In [120]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
from skimage.metrics import peak_signal_noise_ratio as psnr
import cv2
import os
import lpips
import torch

In [121]:
class NerfMetrics():
    def __init__(self):
        super().__init__()

    def computeMetrics(self, imgs_list, gt_list):
        metrics_table = {}
        ssim_list = []
        mse_list = []
        psnr_list = []
        lpips_list = []
        loss_fn_alex = lpips.LPIPS(net='alex')
        #loss_fn_alex.net = loss_fn_alex.net.float()

        for i in range(len(imgs_list)):
            max = (imgs_list[i]*2-1).max()
            min = (imgs_list[i]*2-1).min()
            print(max)
            print(min)
            ssim_ = ssim(imgs_list[i], gt_list[i], data_range=max - min, channel_axis=2)
            mse_ = mean_squared_error(imgs_list[i], gt_list[i])
            psnr_ = psnr(imgs_list[i], gt_list[i])

            ## image from -1 to 1
            lpips_ = loss_fn_alex(torch.Tensor(imgs_list[i]*2-1).permute(2, 0, 1).unsqueeze(0), torch.Tensor(gt_list[i]*2-1).permute(2, 0, 1).unsqueeze(0)).item()

            ssim_list.append(ssim_)
            mse_list.append(mse_)
            psnr_list.append(psnr_)
            lpips_list.append(lpips_)

        metrics_table['ssim'] = ssim_list
        metrics_table['mse'] = mse_list
        metrics_table['psnr'] = psnr_list
        metrics_table['lpips'] = lpips_list

        return metrics_table
    
    def load_imgs(self, img_path, gt_path, dsize=(473,265)):
        imgs_list = []
        gt_list = []
        for filename in os.listdir(img_path):
            img = cv2.imread(os.path.join(img_path,filename))
            if img is not None:
                img = cv2.resize(img, dsize)
                img = img /255.0
                imgs_list.append(img)
        for filename in os.listdir(gt_path):
            gt = cv2.imread(os.path.join(gt_path,filename))
            if gt is not None:
                gt = cv2.resize(gt, dsize)
                gt = gt /255.0
                gt_list.append(gt)

        return imgs_list, gt_list


In [122]:
a = NerfMetrics()

In [136]:
path1 = '/home/fusang/Desktop/Nerf/torch-ngp-la/metrics_test/pre_2'
path2 = '/home/fusang/Desktop/Nerf/torch-ngp-la/metrics_test/gt_2'

In [137]:
img_list, gt_list = a.load_imgs(path1, path2, dsize=(400,400))

In [138]:
for i in range(len(img_list)):
#i = 4
    img = img_list[i]
    gt = gt_list[i]
    cv2.imshow('image',img)  
    cv2.imshow('gt',gt)
    cv2.waitKey(0) # waits until a key is pressed
    cv2.destroyAllWindows()
 

In [139]:
metrcis_table = a.computeMetrics(img_list, gt_list)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/fusang/anaconda3/envs/ngp/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
1.0
-1.0
1.0
-1.0
1.0
-1.0
0.9921568627450981
-0.9843137254901961
1.0
-0.9921568627450981


In [140]:
print(metrcis_table)

{'ssim': [0.6400249983293932, 0.9730945806158201, 0.8059598478914479, 0.7674501822665526, 0.9962529158198454], 'mse': [0.034119878316032304, 0.0003290354030501089, 0.004617814206074584, 0.011923370818915797, 3.599734076637191e-05], 'psnr': [14.669925263547473, 34.827573709797875, 23.35563544545692, 19.236009492011632, 44.43729580709029], 'lpips': [0.3766777217388153, 0.0649774894118309, 0.39152562618255615, 0.5613403916358948, 0.005925499368458986]}


In [141]:
import csv

def createCSV(outputpath, metrcis_table):
    field_names = ['ssim', 'mse', 'psnr', 'lpips']
       
    with open(outputpath, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = field_names)
        writer.writeheader()
        writer.writerow(metrcis_table)


In [119]:
createCSV("./test.csv", metrcis_table)